## no_doc

In [1]:
import json

with open("dataset/rag_truth_train.json", "r") as f:
    train_data = json.load(f)
with open("dataset/rag_truth_dev.json", "r") as f:
    dev_data = json.load(f)
with open("dataset/rag_truth_test.json", "r") as f:
    test_data = json.load(f)

In [2]:
def add_prefix(data):
    for d in data:
        d["text"] = "Please judge the following statement as true or false: " + d["text"]
    return data


train_data = add_prefix(train_data)
dev_data = add_prefix(dev_data)
test_data = add_prefix(test_data)

In [12]:
# task_type: QA, Data2txt, Summary
# Run when you want to train only on specific tasks
task_name = "Summary"
train_data = [d for d in train_data if d["task_type"] == task_name]
dev_data = [d for d in dev_data if d["task_type"] == task_name]
test_data = [d for d in test_data if d["task_type"] == task_name]

In [3]:
from datasets import Dataset, DatasetDict
import pandas as pd

train_df = pd.DataFrame(train_data)
dev_df = pd.DataFrame(dev_data)
test_df = pd.DataFrame(test_data)
train_ds = Dataset.from_pandas(train_df)
dev_ds = Dataset.from_pandas(dev_df)
test_ds = Dataset.from_pandas(test_df)

raw_datasets = DatasetDict({"train": train_ds, "dev":dev_ds, "test": test_ds})
raw_datasets

/gs/fs/tga-arase-student/yamada/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['ref', 'text', 'labels', 'source', 'model', 'task_type', 'source_id'],
        num_rows: 13830
    })
    dev: Dataset({
        features: ['ref', 'text', 'labels', 'source', 'model', 'task_type', 'source_id'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['ref', 'text', 'labels', 'source', 'model', 'task_type', 'source_id'],
        num_rows: 2700
    })
})

In [4]:
from transformers import AutoTokenizer, DataCollatorWithPadding


tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")


def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 2700/2700 [00:00<00:00, 9463.77 examples/s] 


In [5]:
from transformers import AutoModel

base_model = AutoModel.from_pretrained("microsoft/Phi-3.5-mini-instruct")



Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.06s/it]


In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
base_model.to(device)
device

In [7]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    logits = logits[0]
    predictions = np.argmax(logits, axis=-1).tolist()  
    labels = labels.tolist()  

    accuracy = accuracy_score(labels, predictions)
    recall = recall_score(labels, predictions)
    precision = precision_score(labels, predictions)
    f1 = f1_score(labels, predictions)

    return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}

In [ ]:
from transformers import TrainingArguments, Trainer
import torch
from models_phi import NoDocModel

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="steps",
    save_steps=10000,
    learning_rate=1e-6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,
    gradient_accumulation_steps=12,
    logging_dir="./logs",
    report_to="tensorboard",
    optim="adafactor",
)

model = NoDocModel(base_model)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [9]:
trainer.evaluate()

{'eval_loss': 0.9313467741012573,
 'eval_model_preparation_time': 0.0031,
 'eval_accuracy': 0.38571428571428573,
 'eval_recall': 0.8728971962616823,
 'eval_precision': 0.3981244671781756,
 'eval_f1': 0.5468384074941453,
 'eval_runtime': 26.2403,
 'eval_samples_per_second': 48.018,
 'eval_steps_per_second': 12.004}

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

In [ ]:
# evaluate on each task
def create_dev_task(name):
    dev_data2 = [d for d in test_data if d["task_type"] == name]
    dev_df2 = pd.DataFrame(dev_data2)
    dev_ds2 = Dataset.from_pandas(dev_df2)
    tokenized_datasets_task = dev_ds2.map(tokenize_function, batched=True)
    tokenized_datasets_task = tokenized_datasets_task.remove_columns(["text"])
    return tokenized_datasets_task

In [ ]:
dev_qa = create_dev_task("QA")
trainer.evaluate(eval_dataset=dev_qa)

{'eval_loss': 1.2912269830703735,
 'eval_model_preparation_time': 0.0034,
 'eval_accuracy': 0.7977777777777778,
 'eval_recall': 0.5875,
 'eval_precision': 0.44761904761904764,
 'eval_f1': 0.5081081081081081,
 'eval_runtime': 19.817,
 'eval_samples_per_second': 45.416,
 'eval_steps_per_second': 11.354}

In [ ]:
dev_d2t = create_dev_task("Data2txt")
trainer.evaluate(eval_dataset=dev_d2t)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

900


{'eval_loss': 1.7358759641647339,
 'eval_model_preparation_time': 0.0034,
 'eval_accuracy': 0.7822222222222223,
 'eval_recall': 0.8566493955094991,
 'eval_precision': 0.8144499178981938,
 'eval_f1': 0.835016835016835,
 'eval_runtime': 21.8293,
 'eval_samples_per_second': 41.229,
 'eval_steps_per_second': 10.307}

In [ ]:
dev_sum = create_dev_task("Summary")
trainer.evaluate(eval_dataset=dev_sum)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

900


{'eval_loss': 1.951164960861206,
 'eval_model_preparation_time': 0.0034,
 'eval_accuracy': 0.7233333333333334,
 'eval_recall': 0.24509803921568626,
 'eval_precision': 0.3448275862068966,
 'eval_f1': 0.28653295128939826,
 'eval_runtime': 18.196,
 'eval_samples_per_second': 49.461,
 'eval_steps_per_second': 12.365}

In [ ]:
import os

name = "./trained/no_doc_phi"
trainer.save_model(name)
trainer.save_state()
model.save_pretrained(name)

